In [ ]:
try:
  import pydrake
  import underactuated
except ImportError:
  !curl -s https://raw.githubusercontent.com/RussTedrake/underactuated/master/scripts/setup/jupyter_setup.py > jupyter_setup.py
  from jupyter_setup import setup_underactuated
  setup_underactuated()

# Setup matplotlib.  
from IPython import get_ipython
if get_ipython() is not None: get_ipython().run_line_magic("matplotlib", "notebook")

Found my old code at https://github.com/RobotLocomotion/drake/blob/last_sha_with_original_matlab/drake/examples/GridWorld.m


In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

from pydrake.all import DiagramBuilder, LinearSystem, Simulator
from pydrake.systems.controllers import (DynamicProgrammingOptions,
                                         FittedValueIteration)

time_step = 1
# TODO(russt): Support discrete-time systems in the dynamic programming code, and use this properly.
#plant = LinearSystem(A=np.eye(2), B=np.eye(2), C=np.eye(2), D=np.zeros((2,2)), time_period=time_step)
# for now, just cheat because I know how to make the discrete system as a continuous that will be discretized.
plant = LinearSystem(A=np.zeros((2,2)), B=np.eye(2), C=np.eye(2), D=np.zeros((2,2)))
simulator = Simulator(plant)
options = DynamicProgrammingOptions()

xbins = range(0, 21)
ybins = range(0, 21)
state_grid = [set(xbins), set(ybins)]

input_grid = [set([-1, 0, 1]), set([-1, 0, 1])]

goal = [2, 8]

def obstacle(x):
    return x[0]>=6 and x[0]<=8 and x[1]>=4 and x[1]<=7

[X, Y] = np.meshgrid(xbins, ybins)

def draw(iteration, mesh, cost_to_go, policy):
    plt.title("iteration " + str(iteration))
    J = np.reshape(cost_to_go, X.shape)
    im = ax.imshow(J, cmap=cm.jet)

    if plt.get_backend() != u"template":
        fig.canvas.draw()
        plt.pause(1e-10)

    im.remove()

options.visualization_callback = draw

def solve():
    policy, cost_to_go = FittedValueIteration(simulator, cost_function, state_grid,
                                              input_grid, time_step, options)

    J = np.reshape(cost_to_go, X.shape)
    im = ax.imshow(J, cmap=cm.jet)
    return policy
    
def simulate():
    # Animate the resulting policy.
    builder = DiagramBuilder()
    plant = builder.AddSystem(DoubleIntegrator())

    vi_policy = builder.AddSystem(policy)
    builder.Connect(plant.get_output_port(0), vi_policy.get_input_port(0))
    builder.Connect(vi_policy.get_output_port(0), plant.get_input_port(0))

    visualizer = builder.AddSystem(DoubleIntegratorVisualizer())
    builder.Connect(plant.get_output_port(0), visualizer.get_input_port(0))

    diagram = builder.Build()
    simulator = Simulator(diagram)

    simulator.get_mutable_context().SetContinuousState([-10.0, 0.0])

    simulator.AdvanceTo(10.)


In [ ]:
def min_time_cost(context):
    x = context.get_continuous_state_vector().CopyToVector()
    x = np.round(x)
    if obstacle(x):
        return 10
    if np.array_equal(x, goal):
        return 0
    return 1
    
cost_function = min_time_cost
options.convergence_tol = .1;

fig = plt.figure(figsize=(9, 4))
ax = fig.gca()
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_title("Cost-to-Go")

policy = solve()